# SVD Hyperparameter Tuning

This notebook demonstrates how to tune the hyperparameters of the SVD algorithm using GridSearchCV and compares the performance of tuned vs untuned SVD against random recommendations.

## Setup and Imports

First, let's import the necessary libraries and set up our random seeds for reproducibility.

In [1]:
from recsys.MovieLens import MovieLens
from surprise import SVD
from surprise import NormalPredictor
from recsys.Evaluator import Evaluator
from surprise.model_selection import GridSearchCV

import random
import numpy as np

np.random.seed(0)
random.seed(0)

## Load Data

Load the MovieLens dataset and prepare it for evaluation.

In [ ]:
# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = MovieLens.load()

## Grid Search for Best Parameters

We'll use GridSearchCV to find the optimal hyperparameters for the SVD algorithm. We'll search over:
- Number of epochs (20, 30)
- Learning rate (0.005, 0.010)
- Number of factors (50, 100)

In [ ]:
print("Searching for best parameters...")
param_grid = {
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.010],
    'n_factors': [50, 100]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(evaluationData)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print("Best parameters:", gs.best_params['rmse'])

## Initialize Evaluator and Add Algorithms

Now we'll create an evaluator and add three algorithms to compare:
1. SVD with tuned parameters
2. SVD with default parameters
3. Random recommendations (baseline)

In [ ]:
# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# Add tuned SVD
params = gs.best_params['rmse']
SVDtuned = SVD(n_epochs=params['n_epochs'], 
               lr_all=params['lr_all'], 
               n_factors=params['n_factors'])
evaluator.AddAlgorithm(SVDtuned, "SVD - Tuned")

# Add untuned SVD
SVDUntuned = SVD()
evaluator.AddAlgorithm(SVDUntuned, "SVD - Untuned")

# Add random recommendations as baseline
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

## Evaluate Algorithms

Run the evaluation to compare the performance of all algorithms.

In [ ]:
evaluator.Evaluate(False)

## Sample Recommendations

Generate and display some sample recommendations using the evaluated algorithms.

In [ ]:
evaluator.SampleTopNRecs(ml)